In [4]:
import sys
sys.path.append("../src")

In [19]:
import torch
from torch import nn

In [10]:
from models.models import initialize_classification_model
from data.datasets import initialize_datasets

In [9]:
model, input_size = initialize_classification_model('mnist_custom_cnn', 10)
model.load_state_dict(torch.load('../experiments/2022-08-26_01-58-45__model-mnist_custom_cnn_data-mnist_bs-32/model.pt'))
model.eval()

MNISTCustomCNN(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Conv2d(32, 48, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (classification_head): Sequential(
    (0): Linear(in_features=768, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
  (domain_head): Sequential(
    (0): Linear(in_features=768, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [12]:
dataloaders, dataset_sizes = initialize_datasets('mnist', 32, input_size, 1000, 1000)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [18]:
inputs, labels = next(iter(dataloaders['val']))

inputs = inputs.to(device)
labels = labels.to(device)

In [20]:
criterion = nn.CrossEntropyLoss()

In [ ]:
with torch.set_grad_enabled(False):
    outputs, _ = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)

# statistics
running_loss = loss.item() * inputs.size(0)
running_corrects = torch.sum(preds == labels.data).item()